In [1]:
import pandas as pd

df = pd.read_csv('../data/stage_1_train_folds.csv')

In [2]:
df.head()

,image,fold,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0,1,0,0,0,0,0,0
1,ID_00005679d,1,0,0,0,0,0,0
2,ID_00008ce3c,1,0,0,0,0,0,0
3,ID_0000950d7,1,0,0,0,0,0,0
4,ID_0000aee4b,4,0,0,0,0,0,0


In [5]:
from collections import Counter

labels = Counter()

for y in df.values[:, 3:]:
    labels[tuple(y)] += 1

In [7]:
labels.most_common()

[((0, 0, 0, 0, 0), 577154),
 ((0, 0, 0, 0, 1), 28909),
 ((0, 0, 0, 1, 0), 14717),
 ((0, 1, 0, 0, 0), 13932),
 ((0, 0, 1, 0, 0), 8914),
 ((0, 1, 1, 0, 0), 6330),
 ((0, 0, 0, 1, 1), 4654),
 ((0, 1, 0, 1, 0), 3611),
 ((0, 0, 1, 1, 0), 3226),
 ((0, 1, 0, 0, 1), 2952),
 ((0, 1, 0, 1, 1), 2017),
 ((0, 1, 1, 1, 0), 1761),
 ((1, 0, 0, 0, 0), 1497),
 ((0, 0, 1, 0, 1), 1021),
 ((0, 0, 1, 1, 1), 893),
 ((0, 1, 1, 1, 1), 759),
 ((0, 1, 1, 0, 1), 646),
 ((1, 0, 0, 0, 1), 341),
 ((1, 1, 0, 0, 0), 233),
 ((1, 0, 0, 1, 0), 180),
 ((1, 0, 0, 1, 1), 91),
 ((1, 1, 0, 0, 1), 86),
 ((1, 1, 0, 1, 0), 75),
 ((1, 1, 1, 0, 0), 45),
 ((1, 1, 0, 1, 1), 42),
 ((1, 0, 1, 0, 0), 39),
 ((1, 1, 1, 1, 0), 36),
 ((1, 0, 1, 1, 1), 29),
 ((1, 1, 1, 1, 1), 20),
 ((1, 1, 1, 0, 1), 19),
 ((1, 0, 1, 0, 1), 17),
 ((1, 0, 1, 1, 0), 11)]

In [19]:
import numpy as np
import math

def get_sample_weights(labels, weight_per_class):
    sample_weights = [np.max(np.array(weight_per_class)[np.nonzero(lab)[0]]) for lab in labels]
    return sample_weights


def create_class_weight(labels_dict, mu=0.5):
    """ dampened weights. """
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    class_weight_log = dict()

    for key in keys:
        score = total / float(labels_dict[key])
        score_log = math.log(mu * total / float(labels_dict[key]))
        class_weight[key] = round(score, 2) if score > 1.0 else round(1.0, 2)
        class_weight_log[key] = round(score_log, 2) if score_log > 1.0 else round(1.0, 2)

    return class_weight, class_weight_log

weights, damped_weights = create_class_weight(dict(labels))

In [24]:
damped_weights

{(0, 0, 0, 0, 0): 1.0,
 (0, 1, 0, 0, 0): 3.19,
 (0, 0, 0, 1, 1): 4.28,
 (0, 0, 0, 0, 1): 2.46,
 (0, 0, 1, 0, 0): 3.63,
 (0, 0, 0, 1, 0): 3.13,
 (0, 0, 1, 1, 0): 4.65,
 (0, 1, 0, 1, 0): 4.54,
 (0, 1, 0, 0, 1): 4.74,
 (0, 0, 1, 0, 1): 5.8,
 (0, 1, 1, 0, 0): 3.98,
 (0, 1, 1, 0, 1): 6.26,
 (1, 0, 0, 0, 0): 5.42,
 (0, 1, 1, 1, 0): 5.25,
 (0, 1, 0, 1, 1): 5.12,
 (0, 1, 1, 1, 1): 6.1,
 (1, 0, 0, 0, 1): 6.9,
 (0, 0, 1, 1, 1): 5.93,
 (1, 1, 0, 0, 0): 7.28,
 (1, 0, 1, 0, 0): 9.06,
 (1, 1, 0, 1, 1): 8.99,
 (1, 1, 1, 0, 1): 9.78,
 (1, 1, 0, 0, 1): 8.27,
 (1, 1, 1, 0, 0): 8.92,
 (1, 0, 0, 1, 0): 7.54,
 (1, 1, 1, 1, 0): 9.14,
 (1, 1, 0, 1, 0): 8.41,
 (1, 0, 0, 1, 1): 8.22,
 (1, 1, 1, 1, 1): 9.73,
 (1, 0, 1, 1, 0): 10.33,
 (1, 0, 1, 0, 1): 9.9,
 (1, 0, 1, 1, 1): 9.36}

In [21]:
sample_weights = [damped_weights[tuple(label)] for label in df.values[:, 3:]]

In [26]:
df['weight'] = sample_weights

In [28]:
df.to_csv('../data/stage_1_train_folds_weighted.csv', index=False)